In [1]:
from NewCausalscore import *
import pyAgrum.lib.notebook as gnb
import pyAgrum.lib.bn_vs_bn as bnvsbn
%load_ext autoreload
%autoreload 2

In [4]:
nodes = 4
timesteps = 2
values = 2
arcs_ratio = 1.25
datapoints = 1000
islands = 1

In [5]:
generator = Generator(nodes=nodes, timesteps=timesteps, values=values, arcs_ratio=arcs_ratio, islands=islands)
twodbn = generator.generate_structure()

for rep in range(1):
#     if f"structure{num}_{rep}.csv" in os.listdir("/gpfs/home5/jvwijk/outputs/scores3/"):
#         continue

    scores = {}
    true_dbn = generator.generate_DBN(twodbn)

#     true_dbn.saveBIFXML(f"/gpfs/home5/jvwijk/outputs/groundtruths3/structure{num}_{rep}.bifxml")

    generator.generate_data(items=datapoints)

    for subpoints in [int(np.ceil(datapoints/(10**i))) for i in range(2,-1,-1)]:#[100,500]+[int(np.ceil(datapoints/10*i)) for i in range(1,11)]:

        methods = ['HC', 'TABU', 'MMHC', '3OFF2', 'MIIC']
        scorefunctions = ['BDEU', 'AIC', 'BIC', 'K2', 'L2L']

        for alg in methods:
            if alg == "3OFF2" or alg == "MIIC":
                generator.set_learn_method(algorithm=alg, scorefunction=metric)
                generator.learn_causal_structure(subpoints)
                generator.learn_time_structure(subpoints)
                learned = generator.get_structure()
#                 learned.saveBIFXML(f"/gpfs/home5/jvwijk/outputs/structures3/structure{num}_{rep}_{subpoints}_{alg}_constraint.bifxml")

        #         score = {k:[v] for k,v in generator.compare(subpoints).items()}
                score = generator.compare(subpoints)
                score = {k:[v] for k,v in score.items()}

                if scores:
                    for key, value in scores.items():
                        scores[key] = value + score[key]
                else:
                    scores = score
            else:
                for metric in scorefunctions:

                    if alg == 'MMHC' and (metric=='AIC' or metric=="L2L"):
                        continue
                    else:
                        generator.set_learn_method(algorithm=alg, scorefunction=metric)
                        generator.learn_causal_structure(subpoints)
                        generator.learn_time_structure(subpoints)
                        learned = generator.get_structure()
#                         learned.saveBIFXML(f"/gpfs/home5/jvwijk/outputs/structures3/structure{num}_{rep}_{subpoints}_{alg}_{metric}.bifxml")

                #         score = {k:[v] for k,v in generator.compare(subpoints).items()}
                        score = generator.compare(subpoints)
                        score = {k:[v] for k,v in score.items()}

                        if scores:
                            for key, value in scores.items():
                                scores[key] = value + score[key]
                        else:
                            scores = score

In [7]:
pd.DataFrame(scores)

,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,precision,fscore,dist2opt,tp,tn,fp,fn
0,10,4,2,1.25,1,2,HC,BDEU,0.142857,0.285714,0.190476,1.115750,2,37,5,12
1,10,4,2,1.25,1,2,HC,AIC,0.000000,0.000000,0.000000,1.414214,0,41,1,14
2,10,4,2,1.25,1,2,HC,BIC,0.000000,0.000000,0.000000,1.414214,0,42,0,14
3,10,4,2,1.25,1,2,HC,K2,0.071429,0.200000,0.105263,1.225661,1,38,4,13
4,10,4,2,1.25,1,2,HC,L2L,0.000000,0.000000,0.000000,1.414214,0,42,0,14
5,10,4,2,1.25,1,2,TABU,BDEU,0.500000,0.636364,0.560000,0.618249,7,38,4,7
6,10,4,2,1.25,1,2,TABU,AIC,0.000000,0.000000,0.000000,1.414214,0,42,0,14
7,10,4,2,1.25,1,2,TABU,BIC,0.000000,0.000000,0.000000,1.414214,0,42,0,14
8,10,4,2,1.25,1,2,TABU,K2,0.000000,0.000000,0.000000,1.414214,0,37,5,14
9,10,4,2,1.25,1,2,TABU,L2L,0.000000,0.000000,0.000000,1.414214,0,42,0,14
